In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from api.utils import parse_offset, today, get_next_n
from api.data.base import data_path, radar_path
from api.gui.base import View, store
from api.gui.params import curves, bbg_params, spot_params, fwd_params
from api.blpw import BlpQuery
from api.gui.radar import RepeatedTimer
from api import giobhutils as gio
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line

pd.options.plotting.backend = "plotly"

In [8]:
from api.gui.collectors import DataCollector, SpotForward, Futures, Market, Mtgs, FwdMatrix
from api.gui.radar2 import EditableGrid
from api.gui.tables import SpotTable, FutureTable, GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable

In [9]:
bq = BlpQuery(timeout=50000).start()

In [10]:
ccys = [
    'EUR', 
    'USD', 
    'GBP'
]

In [11]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[ccys]

In [32]:
def on_change_update_cob_curves(widget, event, payload, self):
    self.update_cob_curves()
    
    
def on_click_change_timer(widget, event, payload, self):
    self.start_stop_timer()
    

def on_click_refresh(widget, event, payload, self):
    widget.loading = True
    try:
        logging.warning('updating')
        self.refresh()
        logging.warning('updated')
    except Exception as e:
        widget.color = 'error'
        raise e
    else:
        widget.color = 'success'
    finally:
        widget.loading = False


class GlobalConfig(View):
    
    def __init__(self, dc=None, **kwargs):
        self.dc = dc
        super().__init__(**kwargs)
    
    def make_widgets(self, **kwargs):
        refresh_btn = v.Btn(
            fab=True,
            # dark=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-sync'])]
        )
            
        timer_tf = v.TextField(
            v_model=30,
            label='time',
            clearable=True,
            dense=True,
            outlined=True,
            type="number",
            class_="mr-0"

        )

        change_tf = v.TextField(
            v_model='1d',
            label='change',
            clearable=True,
            dense=True,
            outlined=True,
            class_="mr-0"

        )

        timer_btn = v.Btn(
            fab=True,
            # dark=True,
            x_small=True,
            plain=True,
            class_='mt-2 mb-2 mr-2 ml-0',
            children=[v.Icon(children=['mdi-timer'])]
        )
        
        self.refresh_btn = refresh_btn
        self.timer_tf = timer_tf
        self.timer_btn = timer_btn
        self.change_tf = change_tf
        
    
    def make_view(self):
        self.html = w.HTML(self.get_html_text())
        self.param_box = v.Row(
            children=[
                v.Col(
                    cols=1.0,
                    children=[self.timer_tf],
                    class_="my-0 py-0 mr-0 pr-0"
                ),
                v.Col(
                    cols=1.0,
                    children=[self.timer_btn, self.refresh_btn],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=1.0,
                    children=[self.change_tf],
                    class_="my-0 py-0 mr-0 pr-0"
                ),
                v.Col(
                    cols=2,
                    children=[self.html],
                    class_="my-0 py-0"
                ),
            ]
        )
        
        self.view = self.param_box
        
    def link(self, **kwargs):
        self.refresh_btn.on_event(
            'click',
            partial(on_click_refresh, self=self)
        )
        self.timer_btn.on_event(
            'click',
            partial(on_click_change_timer, self=self)
        )

        self.change_tf.on_event(
            'change',
            partial(on_change_update_cob_curves, self=self)
        )
        super().link(**kwargs)
        
    def update_cob_curves(self):
        self.dc.offset = self.change_tf.v_model
    
    def refresh(self, dc=None):
        dc = dc or self.dc
        dc.get_data()
        self.html.value = self.get_html_text(True)
    
    def get_html_text(self, time=None):
        if not time:
            return f'<b>last update: -'
        else:
            return f'<b>last update: {pd.Timestamp.today(): %H:%M:%S}'
        
    def start_stop_timer(self):
        sec = int(self.timer_tf.v_model)
        try:
            rt = self.rt
            if not rt.is_running:
                rt.interval = sec
        except AttributeError:
            self.rt = RepeatedTimer(self.refresh, sec)
            rt = self.rt

        if rt.is_running:
            self.timer_btn.color = None
            rt.stop()
        else:
            self.timer_btn.color = 'success'
            rt.start()

In [33]:
class Monitor(View):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def make_widgets(self, **kwargs):
        pass
    
    def make_view(self):
        self.container = v.Container(class_="ma-0 pa-0", style_=f"background: {bg_color}")
        self.view = self.container

#     @property
#     def refresher(self):
#         return self._refresher
    
#     @refresher.setter
#     def refresher(self, value):
#         self.refresh_view2()
        
    def render(self, **kwargs):
        dc = kwargs.get('dc')
        # self.refresher = True
        
    # def refresh_view2(self):
        # dc = self.dc
        children = [
            v.Row(
                children=[
                    v.Col(
                        cols=3, 
                        children=[
                            w.VBox([
                                w.HBox([usd_fut.get_widget(dc), mtg_usd.get_widget(dc)]), 
                                 w.HBox([
                                     usd_spot.get_widget(dc),  
                                     w.VBox([mkt_equity.get_widget(dc),  mkt_fx.get_widget(dc), mkt_comdty.get_widget(dc)])
                                ])  
                            ])

                        ]
                    ),
                    v.Col(
                        cols=3, 
                        children=[
                            w.VBox([                                
                                w.HBox([eur_fut.get_widget(dc), mtg_eur.get_widget(dc)]), 
                                w.HBox([eur_spot.get_widget(dc), mkt_govt_bond.get_widget(dc), mkt_govt_bond_fut.get_widget(dc)]),
                                # eur_spot2.get_widget(dc)
                            ])
                        ]
                    ),

                    v.Col(
                        cols=3, 
                        children=[
                            w.VBox([                                
                                w.HBox([gbp_fut.get_widget(dc), mtg_gbp.get_widget(dc)]),
                                gbp_spot.get_widget(dc),                                
                            ])

                        ]
                    ),
                    v.Col(
                        cols=3, 
                        children=[
                          

                        ]
                    ),
                ]
            ),            
            v.Row(
                children=[
                    v.Col(cols=4, children=[usd_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[eur_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[gbp_chart.get_widget(dc)]),
                ]
            ),
            # v.Row(children=[v.Col(cols=5, children=[fly_grid.expr_dg.view])]),
            v.Row(
                children=[
                    v.Col(cols=4, children=[mat_usd_live.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_eur_live.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_gbp_live.get_widget(dc)]),
                ]
            ),
            v.Row(
                children=[
                    v.Col(cols=4, children=[mat_usd_chg.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_eur_chg.get_widget(dc)]),
                    v.Col(cols=4, children=[mat_gbp_chg.get_widget(dc)]),
                ]
            )
        ]
        self.container.children = children
        
        
    def link(self, **kwargs):
        super().link(**kwargs)

In [34]:
### Fly Grid

In [35]:
fly_grid = EditableGrid()
# fly_grid.file_tf.v_model = 'test2'
# fly_grid.load_btn.fire_event('click', None)

In [36]:
usd_spot = SpotTable(gen='USD SOFR')
usd_fut = FutureTable(gen='USD SOFR', ccy='USD')
usd_chart = KeyChart(gen='USD SOFR', chart_kwargs=dict(height=300))

In [37]:
eur_spot = SpotTable(gen='EURIBOR A 6M*', title='EURIBOR')
eur_spot2 = SpotTable(gen='EUR EONIA')
eur_fut = FutureTable(gen='EURIBOR A 6M*', title='EURIBOR', ccy='EUR')
eur_chart = KeyChart(gen='EURIBOR A 6M*', chart_kwargs=dict(height=300))

In [38]:
gbp_spot = SpotTable(gen='GBP SONIA')
gbp_fut = FutureTable(gen='GBP SONIA', ccy='GBP')
gbp_chart = KeyChart(gen='GBP SONIA', chart_kwargs=dict(height=300))

In [39]:
mkt_equity = GroupMktTalbe(group='equity')
mkt_fx = GroupMktTalbe(group='fx', pct=False)
mkt_comdty = GroupMktTalbe(group='commodity')

In [40]:
mtg_usd = MtgTable(ccy='USD')
mtg_eur = MtgTable(ccy='EUR')
mtg_gbp = MtgTable(ccy='GBP')

In [41]:
mat_usd_live = MatTable(ccy='USD', curve_type='live', title='Fwd live')
mat_eur_live = MatTable(ccy='EUR', curve_type='live', title='Fwd live')
mat_gbp_live = MatTable(ccy='GBP', curve_type='live', title='Fwd live')

mat_usd_chg = MatTable(ccy='USD', curve_type='change', title='Fwd change')
mat_eur_chg = MatTable(ccy='EUR', curve_type='change', title='Fwd change')
mat_gbp_chg = MatTable(ccy='GBP', curve_type='change', title='Fwd change')

In [42]:
### Govt Bond and Fut

In [43]:
govt_bond_sep_map = [
     'US30Y',
     'UK30Y',
     'GER30Y',
     'FR30Y',
     'ITA30Y',
     'POR30Y',
     # 'SPA30Y'
]

govt_bond_fut_sep_map = [
     'USA',
     'G A',
     'UBA',
     'OATA',
]

In [44]:
mkt_govt_bond = GroupMktTalbe(
    group='govt bonds', 
    pct=False, 
    precision=3,
    change_precision=2,
    styler_kwargs=dict(
        multipliers={'change': 100},
        finalize=custom_styler_row_line,
        finalize_kwargs=dict(mapping=govt_bond_sep_map)
    ),
    field='yield',   
)
mkt_govt_bond_fut = CustomGovtBondFutTable(
    styler_kwargs=dict(
        finalize=custom_styler_row_line,
        finalize_kwargs=dict(mapping=govt_bond_fut_sep_map)
    )
)

In [45]:
### Monitor

In [46]:
m = Monitor()

In [47]:
### Collector

In [48]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    bq=bq,
    collectors={
        'eg': fly_grid,
        'sf': SpotForward(spot_params=spot_params, fwd_params=fwd_params),
        'fut': Futures(),
        'mkt': Market(),
        'mtgs': Mtgs(),
        'mat': FwdMatrix()
    },
    renderers={
        'monitor': m,
    }
)

In [49]:
gc = GlobalConfig(dc=dc)

In [50]:
class Radar(View):

    def __init__(self, curves=None, **kwargs):
        self._curves = curves
        super().__init__(**kwargs)

    def make_widgets(self):
        editable_grid = EditableGrid(curves=self._curves)
        tabs_bars = v.Tabs(
            # dark=True,
            v_model=None,
            children=[
                v.Tab(children=['Radar']),
                v.Tab(children=['Flies']),
                v.Tab(children=['Wathclist'])
            ]
        )

        t1 = v.Container(
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[m.view])])
            ],
            class_='mx-0',
        )
        t2 = v.Container(
            children=[
                v.Row(children=[v.Col(children=[fly_grid.param_box])]),
                v.Row(children=[v.Col(children=[fly_grid.dg])])
            ],
            class_='mx-0',
        )
        
        t3 = v.Container(
            children=[
                v.Row(children=[v.Col(children=[fly_grid.expr_dg.view])]),
            ],
            class_='mx-0',
        )

        tabs_items = v.TabsItems(
            v_model=None,
            # dark=True,
            children=[
                v.TabItem(children=[t1]),
                v.TabItem(children=[t2]),
                v.TabItem(children=[t3])
            ]
        )
        
        self.tabs_bars = tabs_bars
        self.t1 = t1
        self.t2 = t2
        self.t3 = t3
        self.tabs_items = tabs_items

    def make_view(self):
        self.view = v.Card(
            children=[
                self.tabs_bars,
                self.tabs_items
            ]
        )

    def link(self):
        w.jslink((self.tabs_bars, 'v_model'), (self.tabs_items, 'v_model'))
        super().link()

In [51]:
Radar()

Card(children=[Tabs(children=[Tab(children=['Radar']), Tab(children=['Flies']), Tab(children=['Wathclist'])], …

In [52]:
# from ipyflex import FlexLayout
# widgets = { 'Widget 1': w.HTML('<h1> Widget 1</h1>'),
#             'Widget 2': w.HTML('<h1> Widget 2</h1>'),
#             'Widget 3': w.HTML('<h1> Widget 3</h1>'),
#             'Widget 4': w.HTML('<h1> Widget 4</h1>')
#         }
# dashboard = FlexLayout(
#     widgets,
#     header=True,
#     layout_config = {'borderLeft': False, 'borderRight': False, 'enableSection': True},
# )
# dashboard